# **A family of hybrid conjugate gradient method with restart procedure for unconstrained optimizations and image restorations**

En este código se va a hacer una comparación de los Métodos de Gradiente Conjugado No Lineal con distintas variantes para $\beta_k$ según las fórmulas de:
- FR:  Fletcher-Reeves.
- HS:  Hestenes-Stiefel.
- PRP: Polak-Ribière-Polak.
- DY:  Dai-Yuan.
- CD:  Fletcher.
- LS:  Liu-Storey.

Con la variante propuesta en el artículo: **A family of hybrid conjugate gradient method with restart procedure for unconstrained optimizations and image restorations:**
$$
\beta_k^{\text{JYHL}} = \frac{2 \left( \| g_k \|^2 - \theta_k \frac{\|g_k\|}{\|p_{k-1}\|} |g_k^T p_{k-1}| \right)}{g_{k-1}^T \left( g_{k-1} - d_{k-1} \right)}.
$$
**Se realizaron comparaciones para diversos puntos iniciales. Los parámetros usados son los recomendados en el artículo.**

## **1.- Cargamos las librerías necesarias y definimos las funciones de Hartman, Rosenbrock, Beale y Himmelblau.**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(7)

def BACKTRAKING_WOLF(a: float, p: float, delta: float,
                sigma: float, xk: np.array, f, gradf,
                dk: np.array, Nb: int):
    """
    Backtracking line search with the weak Wolfe conditions.
    """
    for i in range(Nb):
        if (f(xk + a*dk) <= f(xk) + delta*a*gradf(xk).T @ dk) and (gradf(xk + a*dk).T @ dk >= sigma*gradf(xk).T @ dk):
            return a, i
        a = p*a
    return a, Nb

def f_Himmelblau(x: np.array):
    return (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 - 7)**2
def grad_Himmelblau(x: np.array):
    x1 = 4*x[0]*(x[0]**2 + x[1] - 11) + 2*(x[0] + x[1]**2 - 7)
    x2 = 2*(x[0]**2 + x[1] - 11) + 4*x[1]*(x[0] + x[1]**2 - 7)
    return np.array([x1,x2], dtype = float)

def f_Beale(x: np.array):
    return (1.5 - x[0] + x[0]*x[1])**2 + (2.25 - x[0] + x[0]*x[1]**2)**2 + (2.625 - x[0] + x[0]*x[1]**3)**2
def grad_Beale(x: np.array):
    x1 = 2*(x[1] - 1)*(1.5 - x[0] + x[0]*x[1]) + 2*(x[1]**2 - 1)*(2.25 - x[0] + x[0]*x[1]**2) + 2*(x[1]**3 - 1)*(2.625 - x[0] + x[0]*x[1]**3)
    x2 = 2*x[0]*(1.5 - x[0] + x[0]*x[1]) + 4*x[0]*x[1]*(2.25 - x[0] + x[0]*x[1]**2) + 6*x[0]*(x[1]**2)*(2.625 - x[0] + x[0]*x[1]**3)
    return np.array([x1,x2], dtype = float)

def f_Rosenbrock(x: np.array):
    n = len(x)
    s = 0
    for i in range(n-1):
        s = s + 100*(x[i+1] - x[i]**2)**2 + (1 - x[i])**2
    return s
def grad_Rosenbrock(x: np.array):
    n = len(x)
    grad = np.zeros(n)
    grad[0] = -400*x[0]*(x[1] - x[0]**2) - 2*(1-x[0])
    grad[n-1] = 200*(x[n-1] - x[n-2]**2)
    for j in range(1,n-1):
        grad[j] = 200*(x[j]-x[j-1]**2) - 400*x[j]*(x[j+1] - x[j]**2) - 2*(1-x[j])
    return np.array(grad, dtype = float)

def f_cuad1(x: np.array, n: int):
    A1 = n*np.eye(n, dtype = float) + np.ones([n,n], dtype = float)
    b1 = np.ones(n, dtype = float)
    return 0.5 * x.T @ A1 @ x - b1.T @ x
def gradf_cuad1(x: np.array, n: int):
    A1 = n*np.eye(n, dtype = float) + np.ones([n,n], dtype = float)
    b1 = np.ones(n, dtype = float)
    return A1 @ x - b1

## **2. Cargamos las funciones.**

In [2]:
def NONLINEAR_CONJUGATE_GRADIENT(xk: np.array, f, gradf, maxiter: int,
                                 tol: float, a_init: float, p: float,
                                 delta: float, sigma: float, Nb: int,
                                 METHOD: str):
    """
    THIS FINDS THE MINIMIZER OF f USING THE NON-LINEAR CONJUGATE GRADIENT
    METHOD WITH THE SELECTED FORMULA.

    Args:
    - xk:      initial guess.
    - f:       function to minimize.
    - gradf:   gradient of the function to minimize.
    - maxiter: maximum number of iterations.
    - tol:     method tolerance.
    - a_init:  initial value for the step size in backtracking.
    - p:       reduction factor for the step size in backtracking.
    - delta:   parameter for the sufficient descent condition.
    - sigma:   parameter for the curvature condition.
    - Nb:      maximum number of iterations in backtracking.
    - METHOD:  method to use for the conjugate gradient.
        - FR:  Fletcher-Reeves.
        - HS:  Hestenes-Stiefel.
        - PRP: Polak-Ribière-Polak.
        - DY:  Dai-Yuan.
        - CD:  Fletcher.
        - LS:  Liu-Storey.
    
    Outputs:
    - xk:  approach to the minimizer of f.
    - gk:  gradient of f at xk.
    - k:   number of iterations.
    - T/F: if the method converged.
    - nr:  number of restarts (bk = 0).
    """
    gk = gradf(xk)
    dk = -gk
    nr = 0
    for k in range(maxiter + 1):
        if np.linalg.norm(gk) < tol:
            return xk, gk, k, True, nr
        ak = BACKTRAKING_WOLF(a = a_init, p = p, delta = delta,
                              sigma = sigma, xk = xk, f = f,
                              gradf = gradf, dk = dk, Nb = Nb)[0]
        xk = xk + ak*dk
        gk_n = gradf(xk)
        if np.abs(gk_n.T @ gk) < 0.2*np.linalg.norm(gk_n)**2:
            if METHOD == 'FR':
                bk = (gk_n.T @ gk_n) / (gk.T @ gk)
            if METHOD == 'HS':
                yk = gk_n - gk
                bk = (gk_n.T @ yk) / (dk.T @ yk)
            if METHOD == 'PRP':
                yk = gk_n - gk
                bk = (gk_n.T @ yk) / (gk.T @ gk)
            if METHOD == 'DY':
                yk = gk_n - gk
                bk = (gk_n.T @ yk) / (dk.T @ yk)
            if METHOD == 'CD':
                bk = -(gk_n.T @ gk_n) / (gk.T @ dk)
            if METHOD == 'LS':
                yk = gk_n - gk
                bk = -(gk_n.T @ yk) / (gk.T @ dk)
        else:
            bk = 0
            nr += 1
        dk = -gk_n + bk*dk
        gk = gk_n
    return xk, gk, maxiter, False, nr

def IJYHL_CONJUGATE_GRADIENT(xk: np.array, f, gradf, maxiter: int,
                             tol: float, a_init: float, p: float,
                             Nb: int, delta: float, sigma: float,
                             zeta: float, mu: float, nu: float):
    """
    THIS FINDS THE MINIMIZER OF f USING THE IJYHL CONJUGATE GRADIENT METHOD.
    
    Args:
    - xk:      initial guess.
    - f:       function to minimize.
    - gradf:   gradient of the function to minimize.
    - maxiter: maximum number of iterations.
    - tol:     method tolerance.
    - a_init:  initial value for the step size in backtracking.
    - p:       reduction factor for the step size in backtracking.
    - Nb:      maximum number of iterations in backtracking.
    - delta:   parameter for the sufficient descent condition.
    - sigma:   parameter for the curvature condition.
    - zeta:    parameter for the restart direction.
    - mu:      parameter for the restart condition.
    - nu:      parameter for the restart condition.

    Returns:
    - xk:  approach to the minimizer of f.
    - gk:  gradient of f at xk.
    - k:   number of iterations.
    - T/F: if the method converged.
    - nr:  number of restarts.
    """
    gk = gradf(xk)
    dk = -gk
    k = 0
    nr = 0
    while k < maxiter:
        if np.linalg.norm(gk) < tol:
            return xk, gk, k, True, nr
        ak = BACKTRAKING_WOLF(a = a_init, p = p, delta = delta,
                              sigma = sigma, xk = xk, f = f,
                              gradf = gradf, dk = dk, Nb = Nb)[0]
        xk = xk + ak * dk
        gk_n = gradf(xk)
        thetak = np.abs(gk_n.T @ dk) / (np.linalg.norm(gk_n)*np.linalg.norm(dk))
        pk = np.copy(gk)
        qk = np.copy(dk)
        betak = (2*(np.linalg.norm(gk_n)**2 - thetak * (np.linalg.norm(gk_n))/np.linalg.norm(pk) * np.abs(gk_n.T @ pk) )) / max(gk.T @ (gk - dk),2*np.linalg.norm(gk)**2)
        if -mu*np.linalg.norm(gk)**2 <= gk_n.T @ dk <= nu*np.linalg.norm(gk)**2:
            dk = -gk_n + betak*dk
        else:       
            dk = -gk_n + zeta * (gk_n.T @ qk)/(np.linalg.norm(qk)**2) * qk
            nr += 1
        gk = gk_n
        k += 1
    return xk, gk, k, False, nr

## **3. Definición de parámetros.**

In [3]:
# DEFININCIÓN DE PARÁMETROS PROPUESTOS POR LOS AUTORES.
N     = 2000   # Número de iteraciones.
tol   = 10**-6 # Tolerancia.
p     = 0.5    # Factor de reducción del tamaño de paso.
delta = 0.01   # Parámetro de la condición de descenso suficiente.
sigma = 0.1    # Parámetro de la condición de curvatura.
Nb    = 500 
zeta  = 0.04   # Parámetro para la dirección de reinicio.
mu    = 0.9    # Parámetro para la condición de reinicio.
nu    = 0.4    # Parámetro para la condición de reinicio.
METDS = ['FR', 'HS', 'PRP', 'DY', 'CD', 'LS']

## **4. Comparación de los métodos para algunos puntos específicos**.

### **4.1. Función de cuadrática 1:** Para $\mathbf{x}=(x_1,x_2, ..., x_n)$

- $f(\mathbf{x}) = \frac{1}{2} \mathbf{x}^\top\mathbf{A}_1\mathbf{x} - \mathbf{b}_1^\top\mathbf{x}$,
  donde $\mathbf{A}_1$ y $\mathbf{b}_1$ están definidas como en el Ejercicio 1.
- $\mathbf{x}_0 = (0,...,0)\in \mathbb{R}^{10}$ 
- $\mathbf{x}_0 = (0,...,0)\in \mathbb{R}^{100}$ 
- $\mathbf{x}_0 = (0,...,0)\in \mathbb{R}^{500}$ 

In [4]:
n = 10
x0 = np.zeros(n, dtype = float)
print("f(x0):       ", f_cuad1(x0, n))
for i in METDS:
    xk, gk, k, conv, nr = NONLINEAR_CONJUGATE_GRADIENT(xk = x0, f = lambda x: f_cuad1(x, n=n),
                                                       gradf = lambda x: gradf_cuad1(x, n=n),
                                                       maxiter = N, tol = tol, a_init = 1, p = p,
                                                       delta = delta, sigma = sigma, Nb = Nb,
                                                       METHOD = i)
    print(i)
    print("ITERACIONES: ", k)
    print("f(xk):       ", f_cuad1(xk, n))
    print("xk:          ", xk[:4], "...", xk[-4:])
    print("NORMA gk:    ", np.linalg.norm(gk))
    print("CONVERGENCIA:", conv)
    print("REINICIOS:   ", nr)
    print("")
xk, gk, k, conv, nr = IJYHL_CONJUGATE_GRADIENT(xk = x0, f = lambda x: f_cuad1(x, n=n),
                                               gradf = lambda x: gradf_cuad1(x, n=n),
                                               maxiter = N, tol = tol, a_init = 1, p = p,
                                               Nb = Nb, delta = delta, sigma = sigma,
                                               zeta = zeta, mu = mu, nu = nu)
print("IJYHL")
print("ITERACIONES: ", k)
print("f(xk):       ", f_cuad1(xk, n))
print("xk:          ", xk[:4], "...", xk[-4:])
print("NORMA gk:    ", np.linalg.norm(gk))
print("CONVERGENCIA:", conv)
print("REINICIOS:   ", nr)

f(x0):        0.0
FR
ITERACIONES:  11
f(xk):        -0.2499999999999858
xk:           [0.05000001 0.05000001 0.05000001 0.05000001] ... [0.05000001 0.05000001 0.05000001 0.05000001]
NORMA gk:     7.539457464619588e-07
CONVERGENCIA: True
REINICIOS:    11

HS
ITERACIONES:  11
f(xk):        -0.2499999999999858
xk:           [0.05000001 0.05000001 0.05000001 0.05000001] ... [0.05000001 0.05000001 0.05000001 0.05000001]
NORMA gk:     7.539457464619588e-07
CONVERGENCIA: True
REINICIOS:    11

PRP
ITERACIONES:  11
f(xk):        -0.2499999999999858
xk:           [0.05000001 0.05000001 0.05000001 0.05000001] ... [0.05000001 0.05000001 0.05000001 0.05000001]
NORMA gk:     7.539457464619588e-07
CONVERGENCIA: True
REINICIOS:    11

DY
ITERACIONES:  11
f(xk):        -0.2499999999999858
xk:           [0.05000001 0.05000001 0.05000001 0.05000001] ... [0.05000001 0.05000001 0.05000001 0.05000001]
NORMA gk:     7.539457464619588e-07
CONVERGENCIA: True
REINICIOS:    11

CD
ITERACIONES:  11
f(xk):       

In [5]:
n = 100
x0 = np.zeros(n, dtype = float)
print("f(x0):       ", f_cuad1(x0, n))
for i in METDS:
    xk, gk, k, conv, nr = NONLINEAR_CONJUGATE_GRADIENT(xk = x0, f = lambda x: f_cuad1(x, n=n),
                                                       gradf = lambda x: gradf_cuad1(x, n=n),
                                                       maxiter = N, tol = tol, a_init = 1, p = p,
                                                       delta = delta, sigma = sigma, Nb = Nb,
                                                       METHOD = i)
    print(i)
    print("ITERACIONES: ", k)
    print("f(xk):       ", f_cuad1(xk, n))
    print("xk:          ", xk[:4], "...", xk[-4:])
    print("NORMA gk:    ", np.linalg.norm(gk))
    print("CONVERGENCIA:", conv)
    print("REINICIOS:   ", nr)
    print("")
xk, gk, k, conv, nr = IJYHL_CONJUGATE_GRADIENT(xk = x0, f = lambda x: f_cuad1(x, n=n),
                                               gradf = lambda x: gradf_cuad1(x, n=n),
                                               maxiter = N, tol = tol, a_init = 1, p = p,
                                               Nb = Nb, delta = delta, sigma = sigma,
                                               zeta = zeta, mu = mu, nu = nu)
print("IJYHL")
print("ITERACIONES: ", k)
print("f(xk):       ", f_cuad1(xk, n))
print("xk:          ", xk[:4], "...", xk[-4:])
print("NORMA gk:    ", np.linalg.norm(gk))
print("CONVERGENCIA:", conv)
print("REINICIOS:   ", nr)

f(x0):        0.0
FR
ITERACIONES:  29
f(xk):        -0.24999999999999828
xk:           [0.005 0.005 0.005 0.005] ... [0.005 0.005 0.005 0.005]
NORMA gk:     5.669611185421531e-07
CONVERGENCIA: True
REINICIOS:    29

HS
ITERACIONES:  29
f(xk):        -0.24999999999999828
xk:           [0.005 0.005 0.005 0.005] ... [0.005 0.005 0.005 0.005]
NORMA gk:     5.669611185421531e-07
CONVERGENCIA: True
REINICIOS:    29

PRP
ITERACIONES:  29
f(xk):        -0.24999999999999828
xk:           [0.005 0.005 0.005 0.005] ... [0.005 0.005 0.005 0.005]
NORMA gk:     5.669611185421531e-07
CONVERGENCIA: True
REINICIOS:    29

DY
ITERACIONES:  29
f(xk):        -0.24999999999999828
xk:           [0.005 0.005 0.005 0.005] ... [0.005 0.005 0.005 0.005]
NORMA gk:     5.669611185421531e-07
CONVERGENCIA: True
REINICIOS:    29

CD
ITERACIONES:  29
f(xk):        -0.24999999999999828
xk:           [0.005 0.005 0.005 0.005] ... [0.005 0.005 0.005 0.005]
NORMA gk:     5.669611185421531e-07
CONVERGENCIA: True
REINICIOS

In [6]:
n = 500
x0 = np.zeros(n, dtype = float)
print("f(x0):       ", f_cuad1(x0, n))
for i in METDS:
    xk, gk, k, conv, nr = NONLINEAR_CONJUGATE_GRADIENT(xk = x0, f = lambda x: f_cuad1(x, n=n),
                                                       gradf = lambda x: gradf_cuad1(x, n=n),
                                                       maxiter = N, tol = tol, a_init = 1, p = p,
                                                       delta = delta, sigma = sigma, Nb = Nb,
                                                       METHOD = i)
    print(i)
    print("ITERACIONES: ", k)
    print("f(xk):       ", f_cuad1(xk, n))
    print("xk:          ", xk[:4], "...", xk[-4:])
    print("NORMA gk:    ", np.linalg.norm(gk))
    print("CONVERGENCIA:", conv)
    print("REINICIOS:   ", nr)
    print("")
xk, gk, k, conv, nr = IJYHL_CONJUGATE_GRADIENT(xk = x0, f = lambda x: f_cuad1(x, n=n),
                                               gradf = lambda x: gradf_cuad1(x, n=n),
                                               maxiter = N, tol = tol, a_init = 1, p = p,
                                               Nb = Nb, delta = delta, sigma = sigma,
                                               zeta = zeta, mu = mu, nu = nu)
print("IJYHL")
print("ITERACIONES: ", k)
print("f(xk):       ", f_cuad1(xk, n))
print("xk:          ", xk[:4], "...", xk[-4:])
print("NORMA gk:    ", np.linalg.norm(gk))
print("CONVERGENCIA:", conv)
print("REINICIOS:   ", nr)

f(x0):        0.0
FR
ITERACIONES:  301
f(xk):        -0.25000000000000494
xk:           [0.001 0.001 0.001 0.001] ... [0.001 0.001 0.001 0.001]
NORMA gk:     2.913095146047013e-07
CONVERGENCIA: True
REINICIOS:    301

HS
ITERACIONES:  301
f(xk):        -0.25000000000000494
xk:           [0.001 0.001 0.001 0.001] ... [0.001 0.001 0.001 0.001]
NORMA gk:     2.913095146047013e-07
CONVERGENCIA: True
REINICIOS:    301

PRP
ITERACIONES:  301
f(xk):        -0.25000000000000494
xk:           [0.001 0.001 0.001 0.001] ... [0.001 0.001 0.001 0.001]
NORMA gk:     2.913095146047013e-07
CONVERGENCIA: True
REINICIOS:    301

DY
ITERACIONES:  301
f(xk):        -0.25000000000000494
xk:           [0.001 0.001 0.001 0.001] ... [0.001 0.001 0.001 0.001]
NORMA gk:     2.913095146047013e-07
CONVERGENCIA: True
REINICIOS:    301

CD
ITERACIONES:  301
f(xk):        -0.25000000000000494
xk:           [0.001 0.001 0.001 0.001] ... [0.001 0.001 0.001 0.001]
NORMA gk:     2.913095146047013e-07
CONVERGENCIA: True


### **4.3. Función de Himmelblau para $\mathbf{x}=(x_1,x_2)$**

$$f(\mathbf{x}) = (x_1^2 + x_2 - 11)^2 + (x_1 + x_2^2 - 7)^2. $$
$x_0=[2,3]$

In [7]:
x0 = np.array([2,3], dtype = float)
n = len(x0)
print("f(x0):       ", f_Himmelblau(x0))
for i in METDS:
    xk, gk, k, conv, nr = NONLINEAR_CONJUGATE_GRADIENT(xk = x0, f = f_Himmelblau, gradf = grad_Himmelblau,
                                                       maxiter = N, tol = tol, a_init = 1, p = p,
                                                       delta = delta, sigma = sigma, Nb = Nb,
                                                       METHOD = i)
    print(i)
    print("ITERACIONES: ", k)
    print("f(xk):       ", f_Himmelblau(xk))
    print("xk:          ", xk)
    print("NORMA gk:    ", np.linalg.norm(gk))
    print("CONVERGENCIA:", conv)
    print("REINICIOS:   ", nr)
    print("")
xk, gk, k, conv, nr = IJYHL_CONJUGATE_GRADIENT(xk = x0, f = f_Himmelblau, gradf = grad_Himmelblau,
                                               maxiter = N, tol = tol, a_init = 1, p = p,
                                               Nb = Nb, delta = delta, sigma = sigma,
                                               zeta = zeta, mu = mu, nu = nu)
print("IJYHL")
print("ITERACIONES: ", k)
print("f(xk):       ", f_Himmelblau(xk))
print("xk:          ", xk)
print("NORMA gk:    ", np.linalg.norm(gk))
print("CONVERGENCIA:", conv)
print("REINICIOS:   ", nr)

f(x0):        32.0
FR
ITERACIONES:  28
f(xk):        2.8547190504579647e-17
xk:           [3. 2.]
NORMA gk:     3.962762538278035e-08
CONVERGENCIA: True
REINICIOS:    26

HS
ITERACIONES:  24
f(xk):        3.7161811384216236e-15
xk:           [3.         2.00000001]
NORMA gk:     7.055921456967452e-07
CONVERGENCIA: True
REINICIOS:    22

PRP
ITERACIONES:  22
f(xk):        1.0603394093115085e-15
xk:           [3.         1.99999999]
NORMA gk:     3.4312415430343865e-07
CONVERGENCIA: True
REINICIOS:    19

DY
ITERACIONES:  24
f(xk):        3.7161811384216236e-15
xk:           [3.         2.00000001]
NORMA gk:     7.055921456967452e-07
CONVERGENCIA: True
REINICIOS:    22

CD
ITERACIONES:  28
f(xk):        2.8547190504579647e-17
xk:           [3. 2.]
NORMA gk:     3.962762538278035e-08
CONVERGENCIA: True
REINICIOS:    26

LS
ITERACIONES:  22
f(xk):        1.0603394093115085e-15
xk:           [3.         1.99999999]
NORMA gk:     3.4312415430343865e-07
CONVERGENCIA: True
REINICIOS:    19

IJ

$x_0=[2,4]$

In [8]:
x0 = np.array([2,4], dtype = float)
n = len(x0)
print("f(x0):       ", f_Himmelblau(x0))
for i in METDS:
    xk, gk, k, conv, nr = NONLINEAR_CONJUGATE_GRADIENT(xk = x0, f = f_Himmelblau, gradf = grad_Himmelblau,
                                                       maxiter = N, tol = tol, a_init = 1, p = p,
                                                       delta = delta, sigma = sigma, Nb = Nb,
                                                       METHOD = i)
    print(i)
    print("ITERACIONES: ", k)
    print("f(xk):       ", f_Himmelblau(xk))
    print("xk:          ", xk)
    print("NORMA gk:    ", np.linalg.norm(gk))
    print("CONVERGENCIA:", conv)
    print("REINICIOS:   ", nr)
    print("")
xk, gk, k, conv, nr = IJYHL_CONJUGATE_GRADIENT(xk = x0, f = f_Himmelblau, gradf = grad_Himmelblau,
                                               maxiter = N, tol = tol, a_init = 1, p = p,
                                               Nb = Nb, delta = delta, sigma = sigma,
                                               zeta = zeta, mu = mu, nu = nu)
print("IJYHL")
print("ITERACIONES: ", k)
print("f(xk):       ", f_Himmelblau(xk))
print("xk:          ", xk)
print("NORMA gk:    ", np.linalg.norm(gk))
print("CONVERGENCIA:", conv)
print("REINICIOS:   ", nr)

f(x0):        130.0
FR
ITERACIONES:  42
f(xk):        2.6508595754176434e-15
xk:           [ 3.58442835 -1.84812653]
NORMA gk:     7.475969996071486e-07
CONVERGENCIA: True
REINICIOS:    41

HS
ITERACIONES:  42
f(xk):        2.5561816132742194e-15
xk:           [ 3.58442835 -1.84812653]
NORMA gk:     7.34125052496262e-07
CONVERGENCIA: True
REINICIOS:    41

PRP
ITERACIONES:  42
f(xk):        2.535696626283605e-15
xk:           [ 3.58442835 -1.84812653]
NORMA gk:     7.311775338809448e-07
CONVERGENCIA: True
REINICIOS:    41

DY
ITERACIONES:  42
f(xk):        2.5561816132742194e-15
xk:           [ 3.58442835 -1.84812653]
NORMA gk:     7.34125052496262e-07
CONVERGENCIA: True
REINICIOS:    41

CD
ITERACIONES:  42
f(xk):        2.6508595754176434e-15
xk:           [ 3.58442835 -1.84812653]
NORMA gk:     7.475969996071486e-07
CONVERGENCIA: True
REINICIOS:    41

LS
ITERACIONES:  42
f(xk):        2.535696626283605e-15
xk:           [ 3.58442835 -1.84812653]
NORMA gk:     7.311775338809448e-07


### **4.4. Función de Beale para $\mathbf{x}=(x_1,x_2)$**

$$f(\mathbf{x}) = (1.5-x_1 + x_1x_2)^2 + (2.25 - x_1 + x_1x_2^2)^2 + (2.625 - x_1 + x_1x_2^3)^2.$$
$x_0=[2,3]$

In [9]:
x0 = np.array([2,3], dtype = float)
n = len(x0)
print("f(x0):       ", f_Beale(x0))
for i in METDS:
    xk, gk, k, conv, nr = NONLINEAR_CONJUGATE_GRADIENT(xk = x0, f = f_Beale, gradf = grad_Beale,
                                                       maxiter = N, tol = tol, a_init = 1, p = p,
                                                       delta = delta, sigma = sigma, Nb = Nb,
                                                       METHOD = i)
    print(i)
    print("ITERACIONES: ", k)
    print("f(xk):       ", f_Beale(xk))
    print("xk:          ", xk)
    print("NORMA gk:    ", np.linalg.norm(gk))
    print("CONVERGENCIA:", conv)
    print("REINICIOS:   ", nr)
    print("")
xk, gk, k, conv, nr = IJYHL_CONJUGATE_GRADIENT(xk = x0, f = f_Beale, gradf = grad_Beale,
                                               maxiter = N, tol = tol, a_init = 1, p = p,
                                               Nb = Nb, delta = delta, sigma = sigma,
                                               zeta = zeta, mu = mu, nu = nu)
print("IJYHL")
print("ITERACIONES: ", k)
print("f(xk):       ", f_Beale(xk))
print("xk:          ", xk)
print("NORMA gk:    ", np.linalg.norm(gk))
print("CONVERGENCIA:", conv)
print("REINICIOS:   ", nr)

f(x0):        3347.203125
FR
ITERACIONES:  171
f(xk):        4.4824573068808293e-13
xk:           [2.99999834 0.49999957]
NORMA gk:     7.757485347447975e-07
CONVERGENCIA: True
REINICIOS:    140

HS
ITERACIONES:  55
f(xk):        2.4354752898569687e-13
xk:           [2.99999878 0.49999971]
NORMA gk:     8.781807369653178e-07
CONVERGENCIA: True
REINICIOS:    43

PRP
ITERACIONES:  97
f(xk):        1.3577389944872002e-14
xk:           [2.99999977 0.49999996]
NORMA gk:     7.279977549685107e-07
CONVERGENCIA: True
REINICIOS:    81

DY
ITERACIONES:  55
f(xk):        2.4354752898569687e-13
xk:           [2.99999878 0.49999971]
NORMA gk:     8.781807369653178e-07
CONVERGENCIA: True
REINICIOS:    43

CD
ITERACIONES:  141
f(xk):        7.706808499444559e-13
xk:           [3.00000219 0.50000053]
NORMA gk:     9.914742174074543e-07
CONVERGENCIA: True
REINICIOS:    113

LS
ITERACIONES:  88
f(xk):        4.472287852839063e-14
xk:           [2.99999951 0.4999999 ]
NORMA gk:     8.537201994038096e-07


### **4.5. Función de Rosenbrock:** Para $\mathbf{x}=(x_1,x_2, ..., x_n)$

$$ f(\mathbf{x}) = \sum_{i=1}^{n-1} \left[100(x_{i+1} - x_i^2)^2 + (1-x_i)^2 \right]
\quad n\geq 2.$$
- $\mathbf{x}_0 = (-1.2, 1.0)\in \mathbb{R}^{2}$  
- $\mathbf{x}_0 = (-1.2, 1.0, ..., -1.2, 1.0) \in \mathbb{R}^{20}$  
- $\mathbf{x}_0 = (-1.2, 1.0, ..., -1.2, 1.0) \in \mathbb{R}^{40}$ 

In [10]:
x0 = np.array([-1.2, 1.0], dtype = float)
n = len(x0)
print("f(x0):       ", f_Rosenbrock(x0))
for i in METDS:
    xk, gk, k, conv, nr = NONLINEAR_CONJUGATE_GRADIENT(xk = x0, f = f_Rosenbrock, gradf = grad_Rosenbrock,
                                                       maxiter = N, tol = tol, a_init = 1, p = p,
                                                       delta = delta, sigma = sigma, Nb = Nb,
                                                       METHOD = i)
    print(i)
    print("ITERACIONES: ", k)
    print("f(xk):       ", f_Rosenbrock(xk))
    print("xk:          ", xk)
    print("NORMA gk:    ", np.linalg.norm(gk))
    print("CONVERGENCIA:", conv)
    print("REINICIOS:   ", nr)
    print("")
xk, gk, k, conv, nr = IJYHL_CONJUGATE_GRADIENT(xk = x0, f = f_Rosenbrock, gradf = grad_Rosenbrock,
                                               maxiter = N, tol = tol, a_init = 1, p = p,
                                               Nb = Nb, delta = delta, sigma = sigma,
                                               zeta = zeta, mu = mu, nu = nu)
print("IJYHL")
print("ITERACIONES: ", k)
print("f(xk):       ", f_Rosenbrock(xk))
print("xk:          ", xk)
print("NORMA gk:    ", np.linalg.norm(gk))
print("CONVERGENCIA:", conv)
print("REINICIOS:   ", nr)

f(x0):        24.199999999999996
FR
ITERACIONES:  2000
f(xk):        4.8699997695794835e-06
xk:           [1.0022068  1.00441906]
NORMA gk:     0.004179577422755096
CONVERGENCIA: False
REINICIOS:    1686

HS
ITERACIONES:  798
f(xk):        4.179405031594755e-13
xk:           [1.00000065 1.00000129]
NORMA gk:     9.790580749730651e-07
CONVERGENCIA: True
REINICIOS:    669

PRP
ITERACIONES:  1302
f(xk):        1.5811876368507815e-14
xk:           [1.00000012 1.00000025]
NORMA gk:     9.99038787351682e-07
CONVERGENCIA: True
REINICIOS:    1259

DY
ITERACIONES:  798
f(xk):        4.179405031594755e-13
xk:           [1.00000065 1.00000129]
NORMA gk:     9.790580749730651e-07
CONVERGENCIA: True
REINICIOS:    669

CD
ITERACIONES:  2000
f(xk):        4.8699997695794835e-06
xk:           [1.0022068  1.00441906]
NORMA gk:     0.004179577422755096
CONVERGENCIA: False
REINICIOS:    1686

LS
ITERACIONES:  1011
f(xk):        4.632001720649427e-14
xk:           [0.99999979 0.99999957]
NORMA gk:     9.7

In [11]:
x0 = np.array([-1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0], dtype = float)
n = len(x0)
print("f(x0):       ", f_Rosenbrock(x0))
for i in METDS:
    xk, gk, k, conv, nr = NONLINEAR_CONJUGATE_GRADIENT(xk = x0, f = f_Rosenbrock, gradf = grad_Rosenbrock,
                                                       maxiter = N, tol = tol, a_init = 1, p = p,
                                                       delta = delta, sigma = sigma, Nb = Nb,
                                                       METHOD = i)
    print(i)
    print("ITERACIONES: ", k)
    print("f(xk):       ", f_Rosenbrock(xk))
    print("xk:          ", xk[:4], "...", xk[-4:])
    print("NORMA gk:    ", np.linalg.norm(gk))
    print("CONVERGENCIA:", conv)
    print("REINICIOS:   ", nr)
    print("")
xk, gk, k, conv, nr = IJYHL_CONJUGATE_GRADIENT(xk = x0, f = f_Rosenbrock, gradf = grad_Rosenbrock,
                                               maxiter = N, tol = tol, a_init = 1, p = p,
                                               Nb = Nb, delta = delta, sigma = sigma,
                                               zeta = zeta, mu = mu, nu = nu)
print("IJYHL")
print("ITERACIONES: ", k)
print("f(xk):       ", f_Rosenbrock(xk))
print("xk:          ", xk[:4], "...", xk[-4:])
print("NORMA gk:    ", np.linalg.norm(gk))
print("CONVERGENCIA:", conv)
print("REINICIOS:   ", nr)

f(x0):        4598.000000000001
FR
ITERACIONES:  655
f(xk):        2.375761737000349e-13
xk:           [1. 1. 1. 1.] ... [1.0000001  1.00000021 1.00000042 1.00000085]
NORMA gk:     9.60433011121481e-07
CONVERGENCIA: True
REINICIOS:    507

HS
ITERACIONES:  953
f(xk):        1.938579986045034e-15
xk:           [1. 1. 1. 1.] ... [0.99999999 0.99999998 0.99999997 0.99999994]
NORMA gk:     8.366403773402635e-07
CONVERGENCIA: True
REINICIOS:    723

PRP
ITERACIONES:  695
f(xk):        9.231232347118253e-14
xk:           [1. 1. 1. 1.] ... [0.99999993 0.99999987 0.99999974 0.99999947]
NORMA gk:     7.919289857586314e-07
CONVERGENCIA: True
REINICIOS:    531

DY
ITERACIONES:  953
f(xk):        1.938579986045034e-15
xk:           [1. 1. 1. 1.] ... [0.99999999 0.99999998 0.99999997 0.99999994]
NORMA gk:     8.366403773402635e-07
CONVERGENCIA: True
REINICIOS:    723

CD
ITERACIONES:  777
f(xk):        3.276999946170261e-13
xk:           [1. 1. 1. 1.] ... [0.99999988 0.99999975 0.9999995  0.9999990

In [12]:
x0 = np.array([-1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0], dtype = float)
n = len(x0)
print("f(x0):       ", f_Rosenbrock(x0))
for i in METDS:
    xk, gk, k, conv, nr = NONLINEAR_CONJUGATE_GRADIENT(xk = x0, f = f_Rosenbrock, gradf = grad_Rosenbrock,
                                                       maxiter = N, tol = tol, a_init = 1, p = p,
                                                       delta = delta, sigma = sigma, Nb = Nb,
                                                       METHOD = i)
    print(i)
    print("ITERACIONES: ", k)
    print("f(xk):       ", f_Rosenbrock(xk))
    print("xk:          ", xk[:4], "...", xk[-4:])
    print("NORMA gk:    ", np.linalg.norm(gk))
    print("CONVERGENCIA:", conv)
    print("REINICIOS:   ", nr)
    print("")
xk, gk, k, conv, nr = IJYHL_CONJUGATE_GRADIENT(xk = x0, f = f_Rosenbrock, gradf = grad_Rosenbrock,
                                               maxiter = N, tol = tol, a_init = 1, p = p,
                                               Nb = Nb, delta = delta, sigma = sigma,
                                               zeta = zeta, mu = mu, nu = nu)
print("IJYHL")
print("ITERACIONES: ", k)
print("f(xk):       ", f_Rosenbrock(xk))
print("xk:          ", xk[:4], "...", xk[-4:])
print("NORMA gk:    ", np.linalg.norm(gk))
print("CONVERGENCIA:", conv)
print("REINICIOS:   ", nr)

f(x0):        9680.000000000002
FR
ITERACIONES:  1078
f(xk):        2.862529124484143e-14
xk:           [1. 1. 1. 1.] ... [1.00000004 1.00000007 1.00000014 1.00000029]
NORMA gk:     8.374461546464992e-07
CONVERGENCIA: True
REINICIOS:    837

HS
ITERACIONES:  2000
f(xk):        18.42416427892163
xk:           [1.00006045 0.99984403 1.00023061 0.99967714] ... [0.01020834 0.01020421 0.01000409 0.00010008]
NORMA gk:     3.340162861360251
CONVERGENCIA: False
REINICIOS:    1545

PRP
ITERACIONES:  1041
f(xk):        3.222880139464832e-15
xk:           [1. 1. 1. 1.] ... [0.99999999 0.99999998 0.99999995 0.9999999 ]
NORMA gk:     8.344566772496873e-07
CONVERGENCIA: True
REINICIOS:    812

DY
ITERACIONES:  2000
f(xk):        18.42416427892163
xk:           [1.00006045 0.99984403 1.00023061 0.99967714] ... [0.01020834 0.01020421 0.01000409 0.00010008]
NORMA gk:     3.340162861360251
CONVERGENCIA: False
REINICIOS:    1545

CD
ITERACIONES:  1796
f(xk):        2.4844540536393238e-14
xk:           [1

## **5. Comparación de los métodos para $20$ puntos aleatorios**.

### **5.1. Función de Himmelblau para $\mathbf{x}=(x_1,x_2)$**

$$f(\mathbf{x}) = (x_1^2 + x_2 - 11)^2 + (x_1 + x_2^2 - 7)^2. $$

In [13]:
CONVERGENCIA_METODOS = np.zeros(len(METDS), dtype = 'float')
ITERACIONES_METODOS = np.zeros(len(METDS), dtype = 'float')
CONVERGENCIA_IJYHL = 0
ITERACIONES_IJYHL = 0
for j in range(20):
    x0 = np.random.uniform(-4, 4, 2)
    for i in METDS:
        xk, gk, k, conv, nr = NONLINEAR_CONJUGATE_GRADIENT(xk = x0, f = f_Himmelblau, gradf = grad_Himmelblau,
                                                           maxiter = N, tol = tol, a_init = 1, p = p,
                                                           delta = delta, sigma = sigma, Nb = Nb,
                                                           METHOD = i)
        if conv == True:
            CONVERGENCIA_METODOS[METDS.index(i)] += 1
        ITERACIONES_METODOS[METDS.index(i)] += k
    xk, gk, k, conv, nr = IJYHL_CONJUGATE_GRADIENT(xk = x0, f = f_Himmelblau, gradf = grad_Himmelblau,
                                                   maxiter = N, tol = tol, a_init = 1, p = p,
                                                   Nb = Nb, delta = delta, sigma = sigma,
                                                   zeta = zeta, mu = mu, nu = nu)
    if conv == True:
        CONVERGENCIA_IJYHL += 1
    ITERACIONES_IJYHL += k
print("         CONV   ITER")
print("FR:      ", CONVERGENCIA_METODOS[0]/20, "", ITERACIONES_METODOS[0]/20)
print("HS:      ", CONVERGENCIA_METODOS[1]/20, "", ITERACIONES_METODOS[1]/20)
print("PRP:     ", CONVERGENCIA_METODOS[2]/20, "", ITERACIONES_METODOS[2]/20)
print("DY:      ", CONVERGENCIA_METODOS[3]/20, "", ITERACIONES_METODOS[3]/20)
print("CD:      ", CONVERGENCIA_METODOS[4]/20, "", ITERACIONES_METODOS[4]/20)
print("LS:      ", CONVERGENCIA_METODOS[5]/20, "", ITERACIONES_METODOS[5]/20)
print("IJYHL:   ", CONVERGENCIA_IJYHL/20, "", ITERACIONES_IJYHL/20)

         CONV   ITER
FR:       0.8  420.0
HS:       0.85  320.95
PRP:      0.8  419.95
DY:       0.85  320.95
CD:       0.8  420.0
LS:       0.8  419.95
IJYHL:    0.85  319.0


### **5.2. Función de Beale para $\mathbf{x}=(x_1,x_2)$**

$$f(\mathbf{x}) = (1.5-x_1 + x_1x_2)^2 + (2.25 - x_1 + x_1x_2^2)^2 + (2.625 - x_1 + x_1x_2^3)^2.$$

In [14]:
CONVERGENCIA_METODOS = np.zeros(len(METDS), dtype = 'float')
ITERACIONES_METODOS = np.zeros(len(METDS), dtype = 'float')
CONVERGENCIA_IJYHL = 0
ITERACIONES_IJYHL = 0
for j in range(20):
    x0 = np.random.uniform(-2, 2, 2)
    for i in METDS:
        xk, gk, k, conv, nr = NONLINEAR_CONJUGATE_GRADIENT(xk = x0, f = f_Beale, gradf = grad_Beale,
                                                           maxiter = N, tol = tol, a_init = 1, p = p,
                                                           delta = delta, sigma = sigma, Nb = Nb,
                                                           METHOD = i)
        if conv == True:
            CONVERGENCIA_METODOS[METDS.index(i)] += 1
        ITERACIONES_METODOS[METDS.index(i)] += k
    xk, gk, k, conv, nr = IJYHL_CONJUGATE_GRADIENT(xk = x0, f = f_Beale, gradf = grad_Beale,
                                                   maxiter = N, tol = tol, a_init = 1, p = p,
                                                   Nb = Nb, delta = delta, sigma = sigma,
                                                   zeta = zeta, mu = mu, nu = nu)
    if conv == True:
        CONVERGENCIA_IJYHL += 1
    ITERACIONES_IJYHL += k
print("         CONV   ITER")
print("FR:      ", CONVERGENCIA_METODOS[0]/20, "", ITERACIONES_METODOS[0]/20)
print("HS:      ", CONVERGENCIA_METODOS[1]/20, "", ITERACIONES_METODOS[1]/20)
print("PRP:     ", CONVERGENCIA_METODOS[2]/20, "", ITERACIONES_METODOS[2]/20)
print("DY:      ", CONVERGENCIA_METODOS[3]/20, "", ITERACIONES_METODOS[3]/20)
print("CD:      ", CONVERGENCIA_METODOS[4]/20, "", ITERACIONES_METODOS[4]/20)
print("LS:      ", CONVERGENCIA_METODOS[5]/20, "", ITERACIONES_METODOS[5]/20)
print("IJYHL:   ", CONVERGENCIA_IJYHL/20, "", ITERACIONES_IJYHL/20)

         CONV   ITER
FR:       0.65  767.05
HS:       0.65  748.05
PRP:      0.7  701.15
DY:       0.65  748.05
CD:       0.7  701.85
LS:       0.7  699.35
IJYHL:    0.75  573.75


### **5.3. Función de Rosenbrock:** Para $\mathbf{x}=(x_1,x_2, ..., x_n)$

$$ f(\mathbf{x}) = \sum_{i=1}^{n-1} \left[100(x_{i+1} - x_i^2)^2 + (1-x_i)^2 \right]
\quad n\geq 2.$$

In [15]:
CONVERGENCIA_METODOS = np.zeros(len(METDS), dtype = 'float')
ITERACIONES_METODOS = np.zeros(len(METDS), dtype = 'float')
CONVERGENCIA_IJYHL = 0
ITERACIONES_IJYHL = 0
for j in range(20):
    x0 = np.random.uniform(-1, 2, 2)
    for i in METDS:
        xk, gk, k, conv, nr = NONLINEAR_CONJUGATE_GRADIENT(xk = x0, f = f_Rosenbrock, gradf = grad_Rosenbrock,
                                                           maxiter = N, tol = tol, a_init = 1, p = p,
                                                           delta = delta, sigma = sigma, Nb = Nb,
                                                           METHOD = i)
        if conv == True:
            CONVERGENCIA_METODOS[METDS.index(i)] += 1
        ITERACIONES_METODOS[METDS.index(i)] += k
    xk, gk, k, conv, nr = IJYHL_CONJUGATE_GRADIENT(xk = x0, f = f_Rosenbrock, gradf = grad_Rosenbrock,
                                                   maxiter = N, tol = tol, a_init = 1, p = p,
                                                   Nb = Nb, delta = delta, sigma = sigma,
                                                   zeta = zeta, mu = mu, nu = nu)
    if conv == True:
        CONVERGENCIA_IJYHL += 1
    ITERACIONES_IJYHL += k
print("         CONV   ITER")
print("FR:      ", CONVERGENCIA_METODOS[0]/20, "", ITERACIONES_METODOS[0]/20)
print("HS:      ", CONVERGENCIA_METODOS[1]/20, "", ITERACIONES_METODOS[1]/20)
print("PRP:     ", CONVERGENCIA_METODOS[2]/20, "", ITERACIONES_METODOS[2]/20)
print("DY:      ", CONVERGENCIA_METODOS[3]/20, "", ITERACIONES_METODOS[3]/20)
print("CD:      ", CONVERGENCIA_METODOS[4]/20, "", ITERACIONES_METODOS[4]/20)
print("LS:      ", CONVERGENCIA_METODOS[5]/20, "", ITERACIONES_METODOS[5]/20)
print("IJYHL:   ", CONVERGENCIA_IJYHL/20, "", ITERACIONES_IJYHL/20)

         CONV   ITER
FR:       0.0  2000.0
HS:       0.95  931.75
PRP:      0.95  883.25
DY:       0.95  931.75
CD:       0.0  2000.0
LS:       0.95  1025.2
IJYHL:    1.0  420.5
